In [1]:
import re
import gurobipy
import cobra
from cobra.flux_analysis.gapfilling import GapFiller
from cobra.io import read_sbml_model, write_sbml_model
from cobra import exceptions
import sys, os
import pickle
from Gapfilling import add_exchange_reactions, add_transport, fake_copy, gapfilling, is_transport, load_query_model, load_template_models 

In [44]:
# Changing "model" variable by "Model" and "templates" by "Templates"
def homology_gapfilling(model, templates, model_obj = None, template_obj = None, use_all_templates = False,
                       integer_threshold = 1e-6, force_exchange = False, force_transport = False, t_all_compounds = False,
                       t_ignore_h = False, value_fraction = 0.8):
    """
    Performs gap filling on a model using homology models as templates.
    """
    Model = load_query_model(model, obj = model_obj)
    Model.solver = 'gurobi'
    Templates = load_template_models(templates, obj = template_obj)
    # this dict will store used models, genes and reactions
    added_reactions = {}
    # initial flux value
    value = Model.optimize().objective_value
    if value == None:
        value = 0.0
    if use_all_templates == False:
        for template in Templates:
            # adding exchange reactions
            if force_exchange == True:
                add_exchange_reactions(Model, template)
            # adding transport reactions
            if force_transport == True:
                add_transport(Model, template, all_compounds = t_all_compounds, ignore_h = t_ignore_h)
            template.solver = 'gurobi'
            # reactions "log"
            log = []
            try:
                # result variable will store the reactions ids
                result = gapfilling(Model, template, integer_threshold = integer_threshold)
                for reaction in result[0]:
                    if reaction.id.startswith("EX_"):
                        log.append((reaction.id, "Exchange reaction"))
                    else:
                        log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
                added_reactions[str(template)] = log
                # Adding reactions to the model
                [Model.add_reaction(reaction.copy()) for reaction in result[0]]
                # Flux will be evaluated here
                new_value = Model.optimize().objective_value
                if new_value != None and new_value > value:
                    value = new_value
                elif new_value == None:
                    continue
                elif new_value != None and new_value == value:
                    break
                elif new_value < value:
                    if new_value >= value * value_fraction:
                        for i in range(len(log)):
                            Model.reactions.log[i][0].lower_bound = 0.
                            Model.reactions.log[i][0].upper_bound = 0.
                        new_value = Model.optimize().objective_value
                        value = new_value
                    else:
                        [Model.remove_reactions(Model.reactions.log[i][0]) for i in range(len(log))]
                        del added_reactions[str(template)]
                        break
                write_sbml_model(Model, filename = "model.tmp")
                Model = read_sbml_model("model.tmp")
                os.remove("model.tmp")
            except RuntimeError:
                print("\n" + str(template) + ": failed to validate gapfilled model, try lowering the integer_threshold")
            except exceptions.Infeasible:
                print("\n" + str(template) + ": gapfilling optimization failed (infeasible)")
        return Model, added_reactions
    else:
        for template in Templates:
            # adding exchange reactions
            if force_exchange == True:
                add_exchange_reactions(Model, template)
            # adding transport reactions
            if force_transport == True:
                add_transport(Model, template, all_compounds = t_all_compounds, ignore_h = t_ignore_h)
            template.solver = 'gurobi'
            log = []
            try:
                result = gapfilling(Model, template, integer_threshold = integer_threshold)   
                for reaction in result[0]:
                    if reaction.id.startswith("EX_"):
                        log.append((reaction.id, "Exchange reaction"))
                    else:
                        log.append((reaction.id, [str(list(reaction.genes)[i]) 
                                              for i in range(len(list(reaction.genes)))]))
                added_reactions[str(template)] = log
                [Model.add_reaction(reaction.copy()) for reaction in result[0]]
                write_sbml_model(Model, filename = "model.tmp")
                Model = read_sbml_model("model.tmp")
                os.remove("model.tmp")
            except RuntimeError:
                print("\n" + str(template) + ": failed to validate gapfilled model, try lowering the integer_threshold")
            except exceptions.Infeasible:
                print("\n" + str(template) + ": gapfilling optimization failed (infeasible)") 
        return Model, added_reactions

In [45]:
iEC1344_C = read_sbml_model("iEC1344_C.xml")
iEC1364_W = read_sbml_model("iEC1364_W.xml")
iJN746 = read_sbml_model("iJN746.xml")

In [46]:
T = [iJN746, iEC1344_C]

In [47]:
x = load_query_model(iEC1364_W)

In [48]:
X, added_reactions = homology_gapfilling(x, T, use_all_templates=True)

Read LP format model from file /tmp/tmpkzsv2ovg.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpaupb0ky5.lp
Reading time = 0.01 seconds
: 1927 rows, 5528 columns, 21688 nonzeros
Read LP format model from file /tmp/tmpwfn0b8ob.lp
Reading time = 0.01 seconds
: 907 rows, 2108 columns, 8802 nonzeros
Read LP format model from file /tmp/tmprjafaqf1.lp
Reading time = 0.01 seconds
: 907 rows, 2108 columns, 8802 nonzeros
Read LP format model from file /tmp/tmpzjvjfb9e.lp
Reading time = 0.01 seconds
: 1927 rows, 5530 columns, 21806 nonzeros
Read LP format model from file /tmp/tmpuntixnjd.lp
Reading time = 0.01 seconds
: 1927 rows, 5530 columns, 21806 nonzeros
Read LP format model from file /tmp/tmp1q10ad59.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros
Read LP format model from file /tmp/tmpo9ywzd1s.lp
Reading time = 0.01 seconds
: 1934 rows, 5452 columns, 21416 nonzeros


In [49]:
added_reactions

{'iJN746': [('BIOMASS_KT_TEMP', [])], 'iEC1344_C': []}

In [43]:
X.optimize().objective_value

0.1876164009738142